<a href="https://colab.research.google.com/github/rvarshini5656/Email_Classification/blob/main/Cbow_email.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import nltk
from nltk.corpus import stopwords
import re
import warnings

warnings.filterwarnings('ignore')



In [ ]:

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stop_words = set(stopwords.words('english'))

# Print the stopwords
print("Stopwords:")
print(stop_words)

Stopwords:
{'what', 'did', 'before', 'after', 'been', 'do', 'have', 'because', 'o', 'not', 'each', 'haven', 'more', 'hers', 'now', 'other', 'that', 'a', "won't", "hadn't", 'y', 'an', 'll', 'few', 'won', "don't", "doesn't", 'needn', "shan't", 'those', 'some', 'mustn', 'then', 'wasn', 'too', "weren't", 'yours', 'below', 'off', 'how', 'further', "isn't", 'there', 'nor', 'up', 'were', 'again', 'can', "needn't", 'this', 'ours', 'yourselves', 'as', "that'll", 'yourself', "couldn't", 'himself', 'be', 'through', 'he', 'myself', 'wouldn', 'most', 'they', 'until', 'no', 'above', "you've", 'if', "wouldn't", 'out', 'for', 'you', 'she', 'their', 'its', "aren't", "shouldn't", 'down', 'by', 'from', 'here', 'any', 'his', 'against', "haven't", "didn't", 'itself', 'm', 'shouldn', 's', 'is', 'just', 'where', "you'll", 're', 'hadn', 'the', 'about', 'her', 'such', "wasn't", 'who', 'd', 'me', 'we', 'are', 'theirs', 'own', 'herself', 'i', "it's", 'ourselves', 'hasn', "mightn't", 'all', 'but', 'at', 'very', '

In [ ]:

data = pd.read_csv('/content/email_classification.csv')



In [ ]:

data.info()
print(data.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179 entries, 0 to 178
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   email   179 non-null    object
 1   label   179 non-null    object
dtypes: object(2)
memory usage: 2.9+ KB
                                               email label
0  Upgrade to our premium plan for exclusive acce...   ham
1  Happy holidays from our team! Wishing you joy ...   ham
2  We're hiring! Check out our career opportuniti...   ham
3  Your Amazon account has been locked. Click her...  spam
4  Your opinion matters! Take our survey and help...   ham


In [ ]:

def clean_text(text):
    text = re.sub('[^a-zA-Z\s]', '', text)
    text = text.lower()
    text = text.split()
    stop_words = set(stopwords.words('english'))
    text = [word for word in text if word not in stop_words]
    return text


In [ ]:

data['clean_email'] = data['email'].apply(clean_text)


In [ ]:

sentences = data['clean_email'].tolist()
cbow_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, sg=0)


In [ ]:

def get_document_vector(text):
    vector = np.zeros(cbow_model.vector_size)
    count = 0
    for word in text:
        if word in cbow_model.wv:
            vector += cbow_model.wv[word]
            count += 1
    if count > 0:
        vector /= count
    return vector


In [ ]:


data['doc_vector'] = data['clean_email'].apply(get_document_vector)


In [ ]:

X = np.array(data['doc_vector'].tolist())
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:

from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train, y_train)


GaussianNB()

In [ ]:

y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred))

Accuracy: 0.92
              precision    recall  f1-score   support

         ham       0.82      1.00      0.90        14
        spam       1.00      0.86      0.93        22

    accuracy                           0.92        36
   macro avg       0.91      0.93      0.92        36
weighted avg       0.93      0.92      0.92        36

